In [2]:
library(data.table)

In [3]:
# source is here: https://docs.google.com/spreadsheets/d/REDACTED/
# just downloaded
files=system(command="ls /moto/ziab/users/jr3950/data/plates\\ AL1-m248_*", intern=TRUE)
files

[1] "/moto/ziab/users/jr3950/data/plates AL1-m248_barn_mice - Plate Layout m103-m153.tsv"
[2] "/moto/ziab/users/jr3950/data/plates AL1-m248_barn_mice - Plate Layout m218-m248.tsv"
[3] "/moto/ziab/users/jr3950/data/plates AL1-m248_barn_mice - Plate Layout m85-m103.tsv" 
[4] "/moto/ziab/users/jr3950/data/plates AL1-m248_barn_mice - Plate layout AL1-m84 .tsv" 
[5] "/moto/ziab/users/jr3950/data/plates AL1-m248_barn_mice - Plate layout m153-m174.tsv"
[6] "/moto/ziab/users/jr3950/data/plates AL1-m248_barn_mice - Plate layout m174-m195.tsv"
[7] "/moto/ziab/users/jr3950/data/plates AL1-m248_barn_mice - Plate layout m195-m218.tsv"

In [37]:
standardize_ids <- function(id) {
  parts <- stringr::str_split(id, "[-_]", simplify = TRUE)
  prefix <- ifelse(stringr::str_detect(parts[1], "m"), "m", "AL")
  num_part1 <- as.integer(gsub("m|AL", "", parts[1]))
  num_part2 <- as.integer(parts[2])
  standardized_id <- sprintf("%s%03d_%02d", prefix, num_part1, num_part2)
  return(standardized_id)
}


In [38]:
df_completo=list()
for(fj in 1:length(files)){
    df = fread(files[fj],header=FALSE)

    row1=which(df$V2 == "row 1")

    unique(df$V1[row1-1])

    df_completo[[fj]]=data.frame(Sample=NA, Pos=NA, Pos2 =NA, Plate=NA, Plate2=NA)
    setDF(df)

    letters=c("A","B","C","D","E","F","G","H")
    for(plate in row1)
        {
        for(letter in 1:length(letters))
            {
            for(row in 2:13)
                {
                Sample = df[(plate+letter),row]
                Pos=paste(letters[letter],(row-1),sep="")
                Pos2=((letter-1)*12)+(row-1)
                Plate=df$V1[plate]
                Plate2=df$V1[plate-1]
                # List of object names
                object_list <- c("Sample", "Pos", "Pos2", "Plate", "Plate2")

                # Loop through each object
                for (object_name in object_list) {
                  # Retrieve the object using get()
                  object <- get(object_name, envir = globalenv())

                  # Check if it is empty and set it to NA if so
                  if (length(object) == 0) {
                    assign(object_name, NA, envir = globalenv())
                  }
                }

                df_completo[[fj]]=bind_rows(df_completo[[fj]],data.frame(Sample=Sample,Pos=Pos,Pos2=Pos2,Plate=Plate,Plate2=Plate2, stringsAsFactors = FALSE))
            }
        }
    }

    df_completo[[fj]]

    unique(df_completo[[fj]]$Plate2)

    df_completo[[fj]]$Plate[df_completo[[fj]]$Plate == 'nov2007m/3']="m3"
    unique(df_completo[[fj]]$Plate)

    library(stringr)

    df_completo[[fj]]$Plate[grepl("AL",df_completo[[fj]]$Plate)]=str_extract(df_completo[[fj]]$Plate[grepl("AL",df_completo[[fj]]$Plate)], "[A][L][0-9]+")

    unique(df_completo[[fj]]$Plate)

    df_completo[[fj]]$Plate2[grepl("AL",df_completo[[fj]]$Plate2)]

    df_completo[[fj]]$Plate[!grepl("AL",df_completo[[fj]]$Plate)]=str_extract(df_completo[[fj]]$Plate[!grepl("AL",df_completo[[fj]]$Plate)], "[m][0-9]+")

    df_completo[[fj]]$Plate2=str_extract(df_completo[[fj]]$Plate2, "[m][0-9]+")

    df_completo[[fj]][is.na(df_completo[[fj]]$Plate2),]

    df_completo[[fj]]=subset(df_completo[[fj]], !is.na(Sample))

    unique(df_completo[[fj]]$Plate2)

    df_completo[[fj]]$Plate_final = df_completo[[fj]]$Plate2
    df_completo[[fj]]$Plate_final[is.na(df_completo[[fj]]$Plate_final)] = df_completo[[fj]]$Plate[is.na(df_completo[[fj]]$Plate_final)]

    print(unique(df_completo[[fj]]$Plate_final))

    df_completo[[fj]]=subset(df_completo[[fj]], !is.na(Plate_final))

    df_completo[[fj]]$Final_ID = paste(df_completo[[fj]]$Plate_final,df_completo[[fj]]$Pos2,sep="_")
    
    df_completo[[fj]]$Final_ID=unlist(lapply(df_completo[[fj]]$Final_ID,standardize_ids))

    #df_completo[[fj]]$Final_ID_2 = str_replace_all(string = df_completo[[fj]]$Final_ID, pattern = "_", replacement = "-")

    df_completo[[fj]]$Sample[df_completo[[fj]]$Sample==""] = NA


    #library(stringr)

    #for(i in 1:nrow(df_completo[[fj]]))
    #    {
    #    df_completo[[fj]]$Final_ID_3[i] = str_replace_all(df_completo[[fj]]$Final_ID[i], "_", "_0")
    #    df_completo[[fj]]$Final_ID_4[i] = str_replace_all(df_completo[[fj]]$Final_ID_2[i], "-", "-0")
    #}
    #print(head(df_completo[[fj]]))
}

 [1] "m103" "m104" "m105" "m106" "m107" "m108" "m109" "m110" "m111" "m112"
[11] "m113" "m114" "m115" "m116" "m117" "m118" "m119" "m120" "m121" "m122"
[21] "m123" "m124" "m125" "m126" "m127" "m128" "m129" "m130" "m131" "m132"
[31] "m133" "m134" "m135" "m136" "m137" "m138" "m139" "m140" "m141" "m142"
[41] "m143" "m144" "m145" "m146" "m147" "m148" "m149" "m150" "m151" "m152"
[51] "m153"
 [1] NA     "m218" "m219" "m220" "m221" "m222" "m223" "m224" "m225" "m226"
[11] "m227" "m228" "m229" "m230" "m231" "m232" "m233" "m234" "m235" "m236"
[21] "m237" "m238" "m239" "m240" "m241" "m242" "m243" "m244" "m245" "m246"
[31] "m247" "m248"
 [1] "m85"  "m86"  "m87"  "m88"  "m89"  "m90"  "m91"  "m92"  "m93"  "m94" 
[11] "m95"  "m96"  "m97"  "m98"  "m99"  "m100" "m101" "m102" "m103"
 [1] "AL1" "AL2" "AL3" "AL4" "AL5" "AL6" "AL7" "m1"  "m2"  "m3"  "m4"  "m5" 
[13] "m6"  "m7"  "m8"  "m9"  "m10" "m11" "m12" "m13" "m14" "m15" "m16" "m17"
[25] "m18" "m19" "m20" "m21" "m22" "m23" "m24" "m25" "m26" "m27" "m28" "

In [39]:
library(dplyr)

In [40]:
DT::datatable(bind_rows(df_completo))

Warning message in instance$preRenderHook(instance):
"It seems your data is too big for client-side DataTables. You may consider server-side processing: https://rstudio.github.io/DT/server.html"


HTML widgets cannot be represented in plain text (need html)

In [41]:
# manual fixes according to Jari's notes

output=bind_rows(df_completo)

output=subset(output, Plate_final != "m228" | !duplicated(paste(Pos2, Plate_final), fromLast = TRUE)) # m228 seems to be listed first with incorrect sample numbers...

In [42]:
write.table(x = unique(output), file = "./plates.tsv", quote=FALSE,sep="\t", col.names = TRUE,row.names = FALSE)